Test a workflow from the Workflow Hub
================================

Before running this notebook, bring up a Workflow Hub instance:

`docker run -d -p 3000:3000 --name seek fairdom/seek:workflow`

Then use the web interface to create a first user (which will also be the admin). This notebook assumes:

* username: simleo
* password: 0123456789

The next step is to enable the workflows feature, which is disabled by default. In the web interface, click the top-right menu, then "Server admin", then "Enable/disable features", then scroll down to "SEEK features" and click "Workflows enabled".

There's a minimal Workflow RO-Crate example in `data/crates/ro-crate-nf-basic`. To be imported in the Workflow Hub, it needs to be zipped with contents at the top level and have a `.crate.zip` extension. Run `bash data/zip_crates.sh` to create the zipped crate.

Now use the web interface to upload the workflow to the Workflow Hub (in the top-left menu, click "Create" and then "Workflow"). The title (`nf-basic`) should be auto-detected by the importer. Associate the workflow to the Default Project (this should be the default option).

This notebook uses the JSON API to retrieve the workflow from the Workflow Hub instance. It then downloads the workflow's RO-Crate, unpacks it and runs `check_nextflow.py` on it (needs the `nextflow` executable in your system path).

JSON API examples are available at https://github.com/seek4science/seekAPIexamples

In [1]:
import os
import pprint
import requests
import shutil
import tempfile
import zipfile
from urllib.request import urlretrieve

In [2]:
pp = pprint.PrettyPrinter(indent=4)
base_url = 'http://localhost:3000'
headers = {
    "Content-type": "application/vnd.api+json",
    "Accept": "application/vnd.api+json",
    "Accept-Charset": "ISO-8859-1"
}
session = requests.Session()
session.headers.update(headers)
session.auth = "simleo", "0123456789"

In [3]:
r = session.get(base_url + "/workflows")
r.raise_for_status()
data = r.json()["data"]
print("ALL WORKFLOWS:")
pp.pprint(data)

ALL WORKFLOWS:
[   {   'attributes': {'title': 'nf-basic'},
        'id': '1',
        'links': {'self': '/workflows/1'},
        'type': 'workflows'}]


In [4]:
wf_id = [_["id"] for _ in data if _["attributes"]["title"] == "nf-basic"][0]
print("wf_id =", wf_id)

wf_id = 1


In [5]:
r = session.get(base_url + "/workflows/%s" % wf_id)
r.raise_for_status()
data = r.json()["data"]
print("WORKFLOW %s:" % wf_id)
pp.pprint(r.json())
print()

WORKFLOW 1:
{   'data': {   'attributes': {   'content_blobs': [   {   'content_type': 'application/zip',
                                                           'link': 'http://localhost:3000/workflows/1/content_blobs/1',
                                                           'md5sum': '1671bb7a677a6ecf9145b5dbfd5d477c',
                                                           'original_filename': 'ro-crate-nf-basic.crate.zip',
                                                           'sha1sum': '3a2933113c5ced60263db002d748129d2fd11fd0',
                                                           'size': 6357,
                                                           'url': None}],
                                  'created_at': '2020-04-09T14:48:08.539Z',
                                  'description': 'Nextflow basic example',
                                  'internals': None,
                                  'latest_version': 1,
                                  'lic

In [6]:
workflow_class = data["attributes"]["workflow_class"]["key"]
print("WORKFLOW CLASS: %s" % workflow_class)
blob = data["attributes"]["content_blobs"][0]
original_filename = blob["original_filename"]
r = session.get(blob["link"])  # this is a full URL
r.raise_for_status()
data = r.json()["data"]
print("CONTENT BLOB FOR %s:" % original_filename)
pp.pprint(r.json())
print()

WORKFLOW CLASS: Nextflow
CONTENT BLOB FOR ro-crate-nf-basic.crate.zip:
{   'data': {   'attributes': {   'content_type': 'application/zip',
                                  'md5sum': '1671bb7a677a6ecf9145b5dbfd5d477c',
                                  'original_filename': 'ro-crate-nf-basic.crate.zip',
                                  'sha1sum': '3a2933113c5ced60263db002d748129d2fd11fd0',
                                  'size': 6357,
                                  'url': None},
                'id': '1',
                'links': {   'download': '/workflows/1/content_blobs/1/download',
                             'self': '/workflows/1/content_blobs/1'},
                'meta': {   'api_version': '0.3',
                            'base_url': 'http://localhost:3000',
                            'created': '2020-04-09T14:47:54.926Z',
                            'modified': '2020-04-09T14:48:08.496Z',
                            'uuid': '04ff3b60-5c9f-0138-a40b-0242ac110002'},
   

In [7]:
wd = tempfile.mkdtemp(prefix="ro_crate_test_")
crate_zip_path = os.path.join(wd, original_filename)
file_url = base_url + data["links"]["download"]
print("downloading workflow RO-Crate to %s" % crate_zip_path)
urlretrieve(file_url, crate_zip_path)

downloading workflow RO-Crate to /tmp/ro_crate_test_a56jv2lq/ro-crate-nf-basic.crate.zip


('/tmp/ro_crate_test_a56jv2lq/ro-crate-nf-basic.crate.zip',
 <http.client.HTTPMessage at 0x7fc6584b9e10>)

In [8]:
crate_dir_bn = os.path.basename(crate_zip_path).split(".", 1)[0]
crate_dir = os.path.join(wd, crate_dir_bn)
with zipfile.ZipFile(crate_zip_path, "r") as zipf:
    zipf.extractall(crate_dir)

In [9]:
from check_nextflow import main

class Args():
    pass

args = Args()
args.crate_dir = crate_dir
main(args)

OK
